## C02 Types, Type Inference, and Stability
### Julia Type System
#### Using types
`::symbol`

In [1]:
using BenchmarkTools

In [2]:
iam(x::Integer) = "an integer"
iam(x::String) = "a string"

function addme(a,b)
    x::Int64 = 2;
    y = (a+b) /x
    return y
end

addme (generic function with 1 method)

In [3]:
iam(1)

"an integer"

In [4]:
iam("Hello")

"a string"

In [5]:
iam(1.0)

MethodError: MethodError: no method matching iam(::Float64)

Closest candidates are:
  iam(!Matched::String)
   @ Main ~/code/Julia-HPC/02-Types.ipynb:2
  iam(!Matched::Integer)
   @ Main ~/code/Julia-HPC/02-Types.ipynb:1


#### Multidispatch
- Dispatch: dispatch is the process of selecting a function to be executed **at runtime**.

In [6]:
sumsqr(x, y) = x^2 + y^2

sumsqr (generic function with 1 method)

In [7]:
sumsqr(1.5, 2 + im)

5.25 + 4.0im

### Abstract types
- **Abstract types** can'have any instantiated values, fileds. can only be the node of the type hierarchy, not its leaves. They represent sets of related types.
    ```julia
    abstract type Number ; end
    abstract type Real     <: Number  ; end
    abstract type FloatingPoint <: Real  ; end
    abstract type Integer  <: Real  ; end
    abstract type Signed   <: Integer  ; end
    abstract type Unsigned <: Integer ; end
    ```

#### Julia's type hierarchy
<img src="https://new-pic-zpp.oss-cn-guangzhou.aliyuncs.com/pic/202401151305571.png" width=450/>

- `Union{ }` type
- `Nothing` type

#### Composite and immutable types
- `struct end`
- `mutable struct end`

In [8]:
struct Pixel
    x::Int64
    y::Int64
    color::Int64
end

In [9]:
p = Pixel(5,5,100)

Pixel(5, 5, 100)

In [10]:
p.x = 10;

ErrorException: setfield!: immutable struct of type Pixel cannot be changed

In [11]:
p.x

5

In [12]:
mutable struct MPixel
    x::Int64
    y::Int64
    color::Int64
end

In [13]:
p = MPixel(5,5,100)
p.x = 10

10

In [14]:
p.x

10

#### Type parameters
`{}`

In [15]:
struct PPixel{T} 
    x::Int64
    y::Int64
    color::T
end

In [16]:
PPixel{Int}(5,5,100)

PPixel{Int64}(5, 5, 100)

In [17]:
typeof(Array{Float64}(undef, 100))

Vector{Float64} (alias for Array{Float64, 1})

In [18]:
Array{Float64} <: Array{Float64, 100}

false

In [19]:
[parse(Float32, string(x)) for x in 1:5]

5-element Vector{Float32}:
 1.0
 2.0
 3.0
 4.0
 5.0

### Type-stability
> The type of return value is dependent only on the types of its arguments and not the values.

对于同一类输入的输出类型是一致的。

In [20]:
function pos(x)
    if x < 0
        return 0;
    else
        return x;
    end
end

pos (generic function with 1 method)

In [21]:
pos(10.0)

10.0

In [22]:
function pos_fixed(x)
    if x < 0
        return zero(x);
    else
        return x;
    end
end

pos_fixed (generic function with 1 method)

In [23]:
pos_fixed(-1.0)

0.0

#### Identifying type stability
- `@code_warntype`   

In [24]:
@code_warntype pos_fixed(2.5)

MethodInstance for pos_fixed(::Float64)
  from pos_fixed(x) @ Main ~/code/Julia-HPC/02-Types.ipynb:1
Arguments
  #self#::Core.Const(pos_fixed)


  x::Float64
Body::Float64


1 ─ %1 = (

x < 0)

::Bool
└──      goto #3 if not %1
2 ─ %3 = Main.zero(x)::Core.Const(0.0)
└──      return %3
3 ─      return x



In [25]:
@code_warntype pos(2.5)

MethodInstance for pos(::Float64)
  from pos(x) @ Main ~/code/Julia-HPC/02-Types.ipynb:1
Arguments
  #self#::Core.Const(pos)
  x::Float64
Body::Union{Float64, Int64}
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return x



#### Loop Variables

In [26]:
function sumsqrtn(n)
    r = 0;
    for i in 1:n
        r = r + sqrt(i)
    end
    return r
end

sumsqrtn (generic function with 1 method)

In [27]:
@code_warntype sumsqrtn(5)

MethodInstance for sumsqrtn(::Int64)
  from sumsqrtn(n) @ Main ~/code/Julia-HPC/02-Types.ipynb:1
Arguments
  #self#::Core.Const(sumsqrtn)
  n::Int64
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  r::Union{Float64, Int64}
  i::Int64
Body::Union{Float64, Int64}
1 ─       (r = 0)
│   %2  = (1:n)

::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = r::Union{Float64, Int64}
│   %11 = Main.sqrt(i)::Float64
│         (r = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return r



In [28]:
function sumsqrtn_fixed(n)
    r = 0.0;
    for i in 1:n
        r = r + sqrt(i)
    end
    return r
end

sumsqrtn_fixed (generic function with 1 method)

In [29]:
@code_warntype sumsqrtn_fixed(5)

MethodInstance for sumsqrtn_fixed(::Int64)
  from sumsqrtn_fixed(n) @ Main ~/code/Julia-HPC/02-Types.ipynb:1
Arguments
  #self#::Core.Const(sumsqrtn_fixed)
  n::Int64
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  r::Float64
  i::Int64
Body::Float64
1 ─       (r = 0.0

)
│   %2  = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = r::Float64
│   %11 = Main.sqrt(i)::Float64
│         (r = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return r



In [30]:
@btime sumsqrtn(1000_000)
@btime sumsqrtn_fixed(1000_000)

  2.233 ms (0 allocations: 0 bytes)


  1.879 ms (0 allocations: 0 bytes)


6.666671664588418e8

In [31]:
function simdsum(x)
    s = 0
    @simd for i in x
        s += i
    end
    return s
end
function simdsum_fixed(x)
    s = zero(eltype(x))
    @simd for i in x
        s += i
    end
    return s
end

simdsum_fixed (generic function with 1 method)

In [32]:
a = rand(Float32, 10^6);

In [33]:
@btime simdsum(a)

  626.468 μs (1 allocation: 16 bytes)


500083.78f0

In [34]:
@btime simdsum_fixed(a)

  73.775 μs (1 allocation: 16 bytes)


500086.2f0

#### Kernel methods and function barriers

In [35]:
function string_zeros(s::AbstractString)
    n = 1000_000;
    x = s == "Int64" ?
        Vector{Int64}(undef, n) :
        Vector{Float64}(undef, n);
    for i in 1:length(x)
        x[i] = 0
    end
    return x
end

string_zeros (generic function with 1 method)

In [36]:
@btime string_zeros("Int64");

  304.971 μs (2 allocations: 7.63 MiB)


In [37]:
function string_zeros_stable(s::AbstractString)
    n = 1000_000;
    x = s == "Int64" ?
        Vector{Int64}(undef, n) :
        Vector{Float64}(undef, n);
   return fill_zeros(x)
end
function fill_zeros(x)
    for i in 1:length(x)
        x[i] = 0
    end
    return x
end

fill_zeros (generic function with 1 method)

In [38]:
@btime string_zeros_stable("Int64");

  417.187 μs (2 allocations: 7.63 MiB)


### Types in storage locations
When we defined a function, we don't need to specify the types of function arguments or local varaibles for performance reasons.
But for things that hod data, such as arrays, dictionaries, or fields in compoite types, it's best to explicity define the type that it will hold.

#### Arrays
It's best to specify the concrete element type in array.

In [39]:
a = Int64[1,2,3,4,5,6,7,8,9,10];
b = Number[1,2,3,4,5,6,7,8,9,10];

In [40]:
function arr_sumsqr(x::Array{T}) where T <: Number
    r = zero(T)
    for i in 1:length(x)
        r += x[i] ^ 2;
    end
    return r
end

arr_sumsqr (generic function with 1 method)

In [41]:
@btime arr_sumsqr(a)
@btime arr_sumsqr(b)

  15.940 ns (0 allocations: 0 bytes)


  278.198 ns (0 allocations: 0 bytes)


385

#### Composite types


In [42]:
abstract type AbstractPoint end
struct Point <: AbstractPoint
    x
    y
end
struct ConcretePoint <: AbstractPoint
    x::Float64
    y::Float64
end

In [43]:
function sumsqr_points(a::Vector{T}) where T <: AbstractPoint
    r = 0.0
    for i in 1:length(a)
        r += a[i].x ^ 2 + a[i].y ^ 2
    end
    return r
end

sumsqr_points (generic function with 1 method)

In [44]:
p_array = [Point(rand(), rand()) for i in 1:1000_000]
cp_array = [ConcretePoint(rand(), rand()) for i in 1:1000_000]

1000000-element Vector{ConcretePoint}:
 ConcretePoint(0.3862741777916191, 0.745399921028031)
 ConcretePoint(0.6154117686785756, 0.17284983645905416)
 ConcretePoint(0.3734149323794227, 0.029913835748310635)
 ConcretePoint(0.49872260603577945, 0.7417812787561356)
 ConcretePoint(0.45384403857072775, 0.5995073401057808)
 ConcretePoint(0.6371867485442682, 0.12743517302198837)
 ConcretePoint(0.9830270034012977, 0.8971483478590909)
 ConcretePoint(0.9186127587992086, 0.58702715261827)
 ConcretePoint(0.647389440606355, 0.21438505986434364)
 ConcretePoint(0.7864818678423127, 0.3909365199242639)
 ⋮
 ConcretePoint(0.2753462090039829, 0.28721988898123474)
 ConcretePoint(0.09905781688078852, 0.5346513851317843)
 ConcretePoint(0.7271682955546223, 0.8542653204677637)
 ConcretePoint(0.44192176841140574, 0.20291853232823853)
 ConcretePoint(0.2729816549064912, 0.7938307162145429)
 ConcretePoint(0.8153464792223171, 0.449155845591817)
 ConcretePoint(0.6824462208450885, 0.08782009676235791)
 ConcretePoint(0

In [45]:
@btime sumsqr_points(p_array)
@btime sumsqr_points(cp_array)

  61.782 ms (4000000 allocations: 61.04 MiB)


  633.053 μs (1 allocation: 16 bytes)


666709.7657388494

#### Parametric composite types
Make the type more flexible

In [46]:
struct PointWithAbstract <: AbstractPoint
    x::AbstractFloat
    y::AbstractFloat
end

In [47]:
struct ParametricPoint{T <: AbstractFloat} <: AbstractPoint
    x::T
    y::T
end

In [48]:
pwa_array = [PointWithAbstract(rand(), rand()) for i in 1:1000_000]
pp_array = [ParametricPoint(rand(), rand()) for i in 1:1000_000]

1000000-element Vector{ParametricPoint{Float64}}:
 ParametricPoint{Float64}(0.5804890623409619, 0.9483743204687298)
 ParametricPoint{Float64}(0.481674657648568, 0.3739821318422739)
 ParametricPoint{Float64}(0.6486971108867966, 0.330093079150098)
 ParametricPoint{Float64}(0.6291623959823697, 0.4485489993526761)
 ParametricPoint{Float64}(0.746841036678508, 0.9875585969983656)
 ParametricPoint{Float64}(0.8028737687073146, 0.45412287816865293)
 ParametricPoint{Float64}(0.24118192176002073, 0.11555036356831883)
 ParametricPoint{Float64}(0.754135492519933, 0.6563666209719407)
 ParametricPoint{Float64}(0.44638349057188065, 0.36819819656388497)
 ParametricPoint{Float64}(0.2886495889366384, 0.0672549810577392)
 ⋮
 ParametricPoint{Float64}(0.20121273942566764, 0.4347436560448147)
 ParametricPoint{Float64}(0.923959243912987, 0.22221716589220775)
 ParametricPoint{Float64}(0.16631275582551308, 0.9543029100493421)
 ParametricPoint{Float64}(0.44272787834737104, 0.7889558200859931)
 ParametricPoint{Fl

In [49]:
@btime sumsqr_points(pwa_array)
@btime sumsqr_points(pp_array)

  72.034 ms (4000000 allocations: 61.04 MiB)


  656.265 μs (1 allocation: 16 bytes)


666304.9114973872